In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset3/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset3/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset3/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset3/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(26283, 128, 6) (26283, 118) (2991, 128, 6) (2991, 118)


In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from matplotlib import pyplot
import tensorflow as tfs
from  tensorflow import keras

In [4]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
x_train = x_train.reshape((x_train.shape[0], n_steps, n_length, n_features))
x_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_features))

In [5]:
from sklearn.model_selection import train_test_split
x_training, x_validation, y_training, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)


In [6]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 28, 64)     1984      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 24, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 20, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, None, 16, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, None, 16, 64)     0

In [7]:
history = model2.fit(x_training, y_training, epochs=200, verbose=True, validation_data=(x_validation, y_validation), batch_size=100)


Epoch 1/200
211/211 [==============================] - 21s 90ms/step - loss: -14746.7783 - accuracy: 0.0301 - val_loss: -83098.6484 - val_accuracy: 0.0282
Epoch 2/200
211/211 [==============================] - 19s 89ms/step - loss: -464629.5000 - accuracy: 0.0286 - val_loss: -1133218.2500 - val_accuracy: 0.0259
Epoch 3/200
211/211 [==============================] - 18s 87ms/step - loss: -2477234.5000 - accuracy: 0.0253 - val_loss: -4316181.0000 - val_accuracy: 0.0224
Epoch 4/200
211/211 [==============================] - 19s 88ms/step - loss: -7100120.0000 - accuracy: 0.0252 - val_loss: -10575127.0000 - val_accuracy: 0.0221
Epoch 5/200
211/211 [==============================] - 19s 88ms/step - loss: -15090214.0000 - accuracy: 0.0275 - val_loss: -20569664.0000 - val_accuracy: 0.0242
Epoch 6/200
211/211 [==============================] - 19s 88ms/step - loss: -27101636.0000 - accuracy: 0.0256 - val_loss: -34803336.0000 - val_accuracy: 0.0297
Epoch 7/200
211/211 [=========================

In [8]:
# evaluate model
loss, accuracy = model2.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.0247
Testing Accuracy: 0.0254
